随着深度学习框架的发展，开发深度学习应用变得越来越便利。

实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。

在本节中，我们将介绍如何使用tensorflow2.0推荐的keras接口更方便地实现线性回归的训练。

## 3.3.1 生成数据集

我们生成与上一节中相同的数据集。

其中features是训练数据特征，labels是标签。

In [1]:
import tensorflow as tf

num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2

features = tf.random.normal(shape=(num_examples, num_inputs), stddev=1)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b

In [2]:
features

<tf.Tensor: shape=(1000, 2), dtype=float32, numpy=
array([[-0.63731885, -1.1652937 ],
       [ 0.13657226,  0.08132705],
       [-0.07855815, -0.12267611],
       ...,
       [-0.9287159 ,  1.4157802 ],
       [-1.186081  ,  0.411551  ],
       [ 0.35595453,  0.37407658]], dtype=float32)>

In [3]:
labels

<tf.Tensor: shape=(1000,), dtype=float32, numpy=
array([ 6.88736057e+00,  4.19663239e+00,  4.45998240e+00,  5.01186180e+00,
        3.53798580e+00, -2.44156313e+00,  9.78432560e+00, -1.96864128e+00,
        3.96932530e+00,  9.15170670e-01,  7.53907108e+00,  2.83587217e+00,
        4.72475624e+00,  1.04746389e+00,  1.58888340e+00,  4.64790726e+00,
        4.32453251e+00,  8.57341957e+00,  1.41668320e-02,  8.54751587e+00,
        3.88034916e+00,  1.45366545e+01,  5.31932163e+00,  6.33130217e+00,
        3.38154936e+00,  6.07999134e+00,  4.12101173e+00,  1.03544407e+01,
        6.49103832e+00,  2.62511754e+00,  1.59098158e+01,  3.67446923e+00,
       -2.32947588e+00,  4.31558037e+00,  1.09279785e+01,  1.89134598e-01,
        1.10005360e+01, -5.96429825e-01,  1.12487297e+01,  1.10023918e+01,
        5.96933508e+00, -1.43006992e+00,  1.84842324e+00,  3.60800076e+00,
        5.31343079e+00,  6.61696529e+00,  3.97103834e+00,  9.60560322e+00,
        6.88169479e-01,  8.58566284e+00,  5.5004544

## 3.3.2 读取数据

虽然 tensorflow 2.0 对于线性回归可以直接拟合，不用再划分数据集，但我们仍学习一下读取数据的方法

In [4]:
from tensorflow import data as tfdata

In [5]:
batch_size = 10
# 将训练数据的特征和标签组合
dataset = tfdata.Dataset.from_tensor_slices((features,labels))
# 随机读取小批量
dataset = dataset.shuffle(buffer_size = num_examples)
dataset = dataset.batch(batch_size)
data_iter = iter(dataset)

In [6]:
data_iter

shuffle 的 buffer_size 参数应大于等于样本数，batch 可以指定 batch_size 的分割大小。

In [7]:
for X,y in data_iter:
    print(X,y)
    break

tf.Tensor(
[[-0.04181116 -1.5717187 ]
 [-0.5428306  -0.88403183]
 [ 0.31573173  1.0711696 ]
 [ 2.247058   -0.19015668]
 [ 1.7474233  -1.3515563 ]
 [-0.6960054   0.35895896]
 [-0.6294383   0.7268995 ]
 [ 0.84762406 -1.374205  ]
 [-0.35927805 -0.84603035]
 [-0.08100699 -0.7944548 ]], shape=(10, 2), dtype=float32) tf.Tensor(
[ 9.460221   6.1200466  1.1894865  9.340649  12.290138   1.5875285
  0.4696648 10.567545   6.357947   6.739132 ], shape=(10,), dtype=float32)


使用iter(dataset)的方式，只能遍历数据集一次，是一种比较 tricky 的写法，为了复刻原书表达才这样写。这里也给出一种在官方文档中推荐的写法：

In [8]:
for ((batch,(X,y))) in enumerate(dataset):
    print(X,y)
    break

tf.Tensor(
[[ 1.5466969  -0.3764781 ]
 [ 0.5507251   0.15342495]
 [-0.52561796 -1.3433663 ]
 [-1.6544685  -0.16097294]
 [ 0.40316167  1.1188632 ]
 [-0.09626045 -1.0387036 ]
 [ 0.40285474 -0.68679583]
 [ 0.15244402 -1.0050431 ]
 [ 0.24057616  0.00977788]
 [ 0.08122712  0.13704109]], shape=(10, 2), dtype=float32) tf.Tensor(
[8.57342   4.779805  7.7162094 1.4383707 1.202188  7.539071  7.340815
 7.9220347 4.6479073 3.8965144], shape=(10,), dtype=float32)


## 3.3.3 定义模型和初始化参数

Tensorflow 2.0推荐使用Keras定义网络，故使用Keras定义网络 我们先定义一个模型变量model，它是一个Sequential实例。 

在Keras中，Sequential实例可以看作是一个串联各个层的容器。

在构造模型时，我们在该容器中依次添加层。 

当给定输入数据时，容器中的每一层将依次推断下一层的输入尺寸。 

重要的一点是，在Keras中我们无须指定每一层输入的形状。 

线性回归，输入层与输出层等效为一层全连接层keras.layers.Dense()。

Keras 中初始化参数由 kernel_initializer 和 bias_initializer 选项分别设置权重和偏置的初始化方式。

我们从 tensorflow 导入 initializers 模块，指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。

偏差参数默认会初始化为零。RandomNormal(stddev=0.01)指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。

偏差参数默认会初始化为零。

In [9]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import initializers as init

model = keras.Sequential()
model.add(layers.Dense(1, kernel_initializer=init.RandomNormal(stddev=0.01)))

## 3.3.4 定义损失函数

Tensoflow在losses模块中提供了各种损失函数和自定义损失函数的基类，并直接使用它的均方误差损失作为模型的损失函数。

In [10]:
from tensorflow import losses

loss = losses.MeanSquaredError()

## 3.3.5 定义优化算法

同样，我们也无须自己实现小批量随机梯度下降算法。

tensorflow.keras.optimizers 模块提供了很多常用的优化算法比如 SGD、Adam 和 RMSProp 等。

下面我们创建一个用于优化model 所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [11]:
from tensorflow.keras import optimizers
trainer = optimizers.SGD(learning_rate=0.03)

## 3.3.6 训练模型

在使用Tensorflow训练模型时，我们通过调用tensorflow.GradientTape记录动态图梯度，执行tape.gradient获得动态图中各变量梯度。

通过 model.trainable_variables 找到需要更新的变量，并用 trainer.apply_gradients 更新权重，完成一步训练。

In [12]:
num_epochs = 3

for epoch in range(1,num_epochs+1):
    for (batch,(X,y)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            l = loss(model(X,training=True),y)
        grads = tape.gradient(l, model.trainable_variables)
        #display(grads)
        #display(model.trainable_variables)
        trainer.apply_gradients(zip(grads,model.trainable_variables))
    l = loss(model(features), labels)
    print("epochs %d, loss: %f" %(epoch,l))

epochs 1, loss: 0.000085
epochs 2, loss: 0.000000
epochs 3, loss: 0.000000


下面我们分别比较学到的模型参数和真实的模型参数。

我们可以通过model的get_weights()来获得其权重（weight）和偏差（bias）。

学到的参数和真实的参数很接近。

In [13]:
model.get_weights()

[array([[ 2.0000002],
        [-3.3999999]], dtype=float32),
 array([4.1999965], dtype=float32)]

In [15]:
true_w

[2, -3.4]

In [16]:
true_b

4.2

# 小结

使用Tensorflow可以更简洁地实现模型。

- tensorflow.data模块提供了有关数据处理的工具
- tensorflow.keras.layers模块定义了大量神经网络的层
- tensorflow.initializers模块定义了各种初始化方法
- tensorflow.optimizers模块提供了模型的各种优化算法。